## Notebook Magic

In [54]:
%matplotlib inline
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Imports

In [188]:
import os
import yaml
import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np

from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import average_precision_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import confusion_matrix
from sklearn.metrics import auc, roc_curve

from sklearn.linear_model import LogisticRegression, Lasso

from sklearn.metrics import f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

from scipy.stats import randint

import itertools

from IPython.display import display
pd.options.display.max_columns = None
pd.options.display.width = 0

In [189]:
df = pd.read_parquet("s3://iefp-unemployment/modelling/modelling.parquet")
df.shape
df.head(10)

,user_id,journey_count,register_date,exit_date,register_movement,exit_movement,register_category,exit_category,register_reason,exit_reason,ute_id,data_movimento,d_birth_date,d_gender,d_age,d_disabled,d_dependents,d_professional_training,d_profession_card,d_school_qualification,d_college_qualification,d_civil_status,d_registration_type,d_nationality,d_parish,d_center,d_subsidy,d_rsi,d_previous_job_experience,d_desired_location,d_desired_work_time,d_desired_contract,i_rvcc_other_entities,i_job_search_techn,i_professional_rvcc,i_rvcc_integrated,i_train_active_life,i_tutoring_in_individual_job_search,i_job_search_assistance,i_pae_cpe_subsidy,i_paper_subsidy_ace,i_paecpe_ace,i_business_and_employment_creation_help,i_invest_young_target_promoter,i_invest_young_another_promoter,i_other_employment_program,i_insertion_companies,i_professional_internships,i_internship_job,i_active_youth_employment,i_reactive,i_arts_and_crafts_train,i_employment-insertion_contract,i_employment_contract_insertion,i_vocational_train_disabilities_cgd,i_vocational_train_disabilities_cgp,i_post-placement_monitoring,i_information_job_eval_orientat,i_integration_stages_disabilities,614,i_open_market_supported_employment,i_learning,i_collective_orientation_session,i_tutoring_in_collective_job_search,i_feg_counseling_actions,i_initial_pedagogical_train_of_trainers,i_continuing_pedagogical_train_of_trainers,i_technological_expertise,i_portuguese_program_for_everyone,i_contin_train_professional_qual_cgp,i_efa_adult_education_cgp,i_technological_specialization_cgp,i_portuguese_program_for_everyone_cgp,i_contin_train_specialization_cgp,i_efa_contin_train_type_b1,i_efa_contin_train_type_b1_b2,i_efa_contin_train_type_b2,i_efa_contin_train_type_b3,i_efa_contin_train_type_b2_b3,i_efa_s3_type_a,i_modular,i_contin_train_modular_extra_catalogue,779,i_efa_n2_vocational_train,i_efa_n3_vocational_train,i_basic_skills_train,i_modular_active_life_cgp,i_personal_development_job_search_techn,i_assertive_communication_job_search_techn,i_entrepreneurship_skills_job_search_techn,i_train_external_entities,i_education,i_professional_internships_extra_iefp,i_pepal_stages_extra_iefp,i_cei_prevpap_extra_iefp,d_desired_job_sector,d_previous_job_sector,success,journey_length
0,108,1,2016-11-10 17:00:31,2018-07-31 20:06:21,43.0,43.0,2.0,3.0,3.0,NaN,108,2016-11-10 17:00:31,1959-10-06,M,57,False,2.0,True,True,2.0,None,C,SIGAE,PT,180908,206,True,False,140.0,C,C,P,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,2017-02-21,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,Industry_Construction_Manufacturing,Industry_Construction_Manufacturing,False,629.0
1,186,2,2018-05-03 15:46:58,2018-07-02 11:13:14,11.0,31.0,2.0,2.0,5.0,19.0,186,2018-05-03 15:46:58,1966-04-28,F,52,False,0.0,True,True,3.0,None,C,SIGAE,PT,021203,401,True,True,36.0,C,C,P,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,2018-05-09,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,Consumer_Services_Workers,Unskilled workers,True,60.0
2,304,1,2017-01-17 09:51:43,2017-11-21 16:46:03,11.0,31.0,2.0,2.0,7.0,11.0,304,2017-01-17 09:51:43,1962-06-28,M,54,False,2.0,True,True,NaN,None,C,SIGAE,PT,130905,144,True,True,360.0,N,C,P,NaT,2017-05-29,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,Industry_Construction_Manufacturing,Industry_Construction_Manufacturing,False,309.0
3,724,4,2018-04-18 10:58:08,2018-12-19 11:02:11,11.0,31.0,2.0,2.0,15.0,44.0,724,2018-04-18 10:58:08,1978-12-13,F,39,False,1.0,True,True,2.0,None,D,SIGAE,PT,180525,138,True,True,4.0,C,C,P,

In [42]:
# Intervention feature preparation

interv_cols = [col for col in df.columns if "i_" in col]
X = df[interv_cols].copy()

# Strip col names
#X.columns = [col.replace("i_", "") for col in X.columns]

# Make df boolean
X = (X.notna()).astype('int')

# Filter for frequent interventions
frequent_i = X.mean()[X.mean() > 0.01].index.tolist()
X = X[frequent_i]

In [43]:
X.head(10)

,i_job_search_techn,i_train_active_life,i_tutoring_in_individual_job_search,i_professional_internships,i_internship_job,i_employment-insertion_contract,i_employment_contract_insertion,i_information_job_eval_orientat,i_collective_orientation_session,i_tutoring_in_collective_job_search,i_efa_s3_type_a,i_modular,i_efa_n3_vocational_train,i_assertive_communication_job_search_techn,i_entrepreneurship_skills_job_search_techn,i_train_external_entities
0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0
5,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
7,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
9,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


In [47]:
len(X.columns.tolist())

16

In [49]:
# Demographics prep

dems = ["d_age", "d_gender", "d_civil_status", "d_rsi", "d_desired_work_time", "d_desired_contract",
       "d_school_qualification", "d_college_qualification", "d_disabled", "d_subsidy", "d_previous_job_sector",
       "d_desired_job_sector", "d_previous_job_experience"]

#Filling NAs!!


X[dems] = df[dems].fillna(0)
X["register_month"] = df.register_date.dt.month
X["register_year"] = df.register_date.dt.year
X = pd.get_dummies(X, drop_first=True, dummy_na=True)
X.head()

# Set output variable

Y = df["success"]

In [51]:
X.columns.tolist()

['i_job_search_techn',
 'i_train_active_life',
 'i_tutoring_in_individual_job_search',
 'i_professional_internships',
 'i_internship_job',
 'i_employment-insertion_contract',
 'i_employment_contract_insertion',
 'i_information_job_eval_orientat',
 'i_collective_orientation_session',
 'i_tutoring_in_collective_job_search',
 'i_efa_s3_type_a',
 'i_modular',
 'i_efa_n3_vocational_train',
 'i_assertive_communication_job_search_techn',
 'i_entrepreneurship_skills_job_search_techn',
 'i_train_external_entities',
 'd_age',
 'd_rsi',
 'd_school_qualification',
 'd_disabled',
 'd_subsidy',
 'd_previous_job_experience',
 'register_month',
 'register_year',
 'd_gender_M',
 'd_gender_nan',
 'd_civil_status_D',
 'd_civil_status_J',
 'd_civil_status_O',
 'd_civil_status_S',
 'd_civil_status_U',
 'd_civil_status_V',
 'd_civil_status_nan',
 'd_desired_work_time_P',
 'd_desired_work_time_nan',
 'd_desired_contract_P',
 'd_desired_contract_T',
 'd_desired_contract_nan',
 'd_college_qualification_Agricul

In [52]:
# Test/Train split


X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.4, random_state=0)

In [55]:
# Prepare Random Forest pipeline

rf_pipeline = Pipeline([
    ('rf', RandomForestClassifier(n_estimators=50, n_jobs=-1, random_state=0)),
])

'''
# Prepare Random Forest pipeline with scaling

rf__scale_pipeline = Pipeline([
    ('scale', StandardScaler()),
    ('rf', RandomForestClassifier(n_estimators=50, n_jobs=-1, random_state=0))
])
'''

"\n# Prepare Random Forest pipeline with scaling\n\nrf__scale_pipeline = Pipeline([\n    ('scale', StandardScaler()),\n    ('rf', RandomForestClassifier(n_estimators=50, n_jobs=-1, random_state=0))\n])\n"

In [56]:
# Grid search

big_param_grid = [
        {'rf__n_estimators': [1,10,100,1000,10000],
         'rf__max_depth': [1,5,10,20,50,100],
         'rf__max_features': ['sqrt','log2'],
         'rf__min_samples_split': [2,5,10],
         'rf__n_jobs': [-1]
        },
    ]

param_grid = [
        {'rf__n_estimators': [100,1000],
         'rf__max_depth': [2, 3]
        },
    ]


rf_grid_search = GridSearchCV(rf_pipeline, param_grid, cv=3, refit=True)

rf_grid_search.fit(X_train, y_train)

rf_final_model = rf_grid_search.best_estimator_

print(confusion_matrix(y_test, rf_final_model.predict(X_test)))

print(accuracy_score(y_test, rf_final_model.predict(X_test)))

[[23494  5106]
 [15634 10922]]
0.6239756327507433


In [190]:
# Permutations

print(rf_final_model.predict(X_test.head(10)))
print(y_test.head(10))

[False False False False False False False False  True False]
38364     False
110441    False
79466     False
47054      True
25699      True
118797     True
109361     True
51963     False
51344      True
68336      True
Name: success, dtype: bool


In [191]:
combinations = [c for c in itertools.product([0, 1], repeat=16) if sum(c) <= 5]

In [198]:
combo_df = pd.DataFrame(combinations)

In [200]:
combo_df.shape

(6885, 16)

In [201]:
user_data = X_test.head(1)

In [202]:
user_data

,i_job_search_techn,i_train_active_life,i_tutoring_in_individual_job_search,i_professional_internships,i_internship_job,i_employment-insertion_contract,i_employment_contract_insertion,i_information_job_eval_orientat,i_collective_orientation_session,i_tutoring_in_collective_job_search,i_efa_s3_type_a,i_modular,i_efa_n3_vocational_train,i_assertive_communication_job_search_techn,i_entrepreneurship_skills_job_search_techn,i_train_external_entities,d_age,d_rsi,d_school_qualification,d_disabled,d_subsidy,d_previous_job_experience,register_month,register_year,d_gender_M,d_gender_nan,d_civil_status_D,d_civil_status_J,d_civil_status_O,d_civil_status_S,d_civil_status_U,d_civil_status_V,d_civil_status_nan,d_desired_work_time_P,d_desired_work_time_nan,d_desired_contract_P,d_desired_contract_T,d_desired_contract_nan,d_college_qualification_Agriculture,d_college_qualification_Art_Humanities,d_college_qualification_Education,d_college_qualification_Engineering_Manufacturing_Construction,d_college_qualification_General_Programs,d_college_qualification_Health_Social_Work,d_college_qualification_Science_Mathematics_Computing,d_college_qualification_Services,d_college_qualification_Social_Sciences_Business_Law,d_college_qualification_Unknown,d_college_qualification_nan,d_previous_job_sector_Admin_Staff,d_previous_job_sector_Agriculture,d_previous_job_sector_Consumer_Services_Workers,d_previous_job_sector_Executive_Management,d_previous_job_sector_Industry_Construction_Manufacturing,d_previous_job_sector_Machinery_Operators_Transportation,d_previous_job_sector_Scientists_Experts,d_previous_job_sector_Technicians_Mid_Professionals,d_previous_job_sector_Unskilled workers,d_previous_job_sector_nan,d_desired_job_sector_Admin_Staff,d_desired_job_sector_Agriculture,d_desired_job_sector_Consumer_Services_Workers,d_desired_job_sector_Executive_Management,d_desired_job_sector_Industry_Construction_Manufacturing,d_desired_job_sector_Machinery_Operators_Transportation,d_desired_job_sector_Scientists_Experts,d_desired_job_sector_Technicians_Mid_Professionals,d_desired_job_sector_Unskilled workers,d_desired_job_sector_nan
38364,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,33,True,2.0,False,True,149.0,11,2016,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0


In [203]:
user_data_fixed = user_data.drop(user_data.iloc[:,0:16], axis=1)

In [204]:
user_data_fixed

,d_age,d_rsi,d_school_qualification,d_disabled,d_subsidy,d_previous_job_experience,register_month,register_year,d_gender_M,d_gender_nan,d_civil_status_D,d_civil_status_J,d_civil_status_O,d_civil_status_S,d_civil_status_U,d_civil_status_V,d_civil_status_nan,d_desired_work_time_P,d_desired_work_time_nan,d_desired_contract_P,d_desired_contract_T,d_desired_contract_nan,d_college_qualification_Agriculture,d_college_qualification_Art_Humanities,d_college_qualification_Education,d_college_qualification_Engineering_Manufacturing_Construction,d_college_qualification_General_Programs,d_college_qualification_Health_Social_Work,d_college_qualification_Science_Mathematics_Computing,d_college_qualification_Services,d_college_qualification_Social_Sciences_Business_Law,d_college_qualification_Unknown,d_college_qualification_nan,d_previous_job_sector_Admin_Staff,d_previous_job_sector_Agriculture,d_previous_job_sector_Consumer_Services_Workers,d_previous_job_sector_Executive_Management,d_previous_job_sector_Industry_Construction_Manufacturing,d_previous_job_sector_Machinery_Operators_Transportation,d_previous_job_sector_Scientists_Experts,d_previous_job_sector_Technicians_Mid_Professionals,d_previous_job_sector_Unskilled workers,d_previous_job_sector_nan,d_desired_job_sector_Admin_Staff,d_desired_job_sector_Agriculture,d_desired_job_sector_Consumer_Services_Workers,d_desired_job_sector_Executive_Management,d_desired_job_sector_Industry_Construction_Manufacturing,d_desired_job_sector_Machinery_Operators_Transportation,d_desired_job_sector_Scientists_Experts,d_desired_job_sector_Technicians_Mid_Professionals,d_desired_job_sector_Unskilled workers,d_desired_job_sector_nan
38364,33,True,2.0,False,True,149.0,11,2016,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0


In [205]:
user_data_fixed.shape

(1, 53)

In [206]:
user_fixed_df = pd.DataFrame(np.tile(user_data_fixed.values, len(combo_df.index)).reshape(-1,len(user_data_fixed.columns)), 
                   columns=user_data_fixed.columns)

In [208]:
user_fixed_df.shape

(6885, 53)

In [209]:
predict_df = combo_df.join(user_fixed_df)

In [210]:
predict_df.shape

(6885, 69)

In [211]:
probabilities = pd.DataFrame(rf_final_model.predict_proba(predict_df))

In [212]:
probabilities.columns = ["unsuccessful", "successful"]

In [213]:
results_df = predict_df.join(probabilities)

In [214]:
results_df.iloc[:,0:16].columns = ['i_job_search_techn',
 'i_train_active_life',
 'i_tutoring_in_individual_job_search',
 'i_professional_internships',
 'i_internship_job',
 'i_employment-insertion_contract',
 'i_employment_contract_insertion',
 'i_information_job_eval_orientat',
 'i_collective_orientation_session',
 'i_tutoring_in_collective_job_search',
 'i_efa_s3_type_a',
 'i_modular',
 'i_efa_n3_vocational_train',
 'i_assertive_communication_job_search_techn',
 'i_entrepreneurship_skills_job_search_techn',
 'i_train_external_entities']

In [215]:
display(results_df.sort_values(by=['successful'], ascending=False))

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,d_age,d_rsi,d_school_qualification,d_disabled,d_subsidy,d_previous_job_experience,register_month,register_year,d_gender_M,d_gender_nan,d_civil_status_D,d_civil_status_J,d_civil_status_O,d_civil_status_S,d_civil_status_U,d_civil_status_V,d_civil_status_nan,d_desired_work_time_P,d_desired_work_time_nan,d_desired_contract_P,d_desired_contract_T,d_desired_contract_nan,d_college_qualification_Agriculture,d_college_qualification_Art_Humanities,d_college_qualification_Education,d_college_qualification_Engineering_Manufacturing_Construction,d_college_qualification_General_Programs,d_college_qualification_Health_Social_Work,d_college_qualification_Science_Mathematics_Computing,d_college_qualification_Services,d_college_qualification_Social_Sciences_Business_Law,d_college_qualification_Unknown,d_college_qualification_nan,d_previous_job_sector_Admin_Staff,d_previous_job_sector_Agriculture,d_previous_job_sector_Consumer_Services_Workers,d_previous_job_sector_Executive_Management,d_previous_job_sector_Industry_Construction_Manufacturing,d_previous_job_sector_Machinery_Operators_Transportation,d_previous_job_sector_Scientists_Experts,d_previous_job_sector_Technicians_Mid_Professionals,d_previous_job_sector_Unskilled workers,d_previous_job_sector_nan,d_desired_job_sector_Admin_Staff,d_desired_job_sector_Agriculture,d_desired_job_sector_Consumer_Services_Workers,d_desired_job_sector_Executive_Management,d_desired_job_sector_Industry_Construction_Manufacturing,d_desired_job_sector_Machinery_Operators_Transportation,d_desired_job_sector_Scientists_Experts,d_desired_job_sector_Technicians_Mid_Professionals,d_desired_job_sector_Unskilled workers,d_desired_job_sector_nan,unsuccessful,successful
2376,0,0,0,1,1,1,1,0,0,0,1,0,0,0,0,0,33,True,2,False,True,149,11,2016,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0.363014,0.636986
2334,0,0,0,1,1,1,0,0,0,0,0,0,1,1,0,0,33,True,2,False,True,149,11,2016,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0.365778,0.634222
2330,0,0,0,1,1,1,0,0,0,0,0,0,0,1,1,0,33,True,2,False,True,149,11,2016,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0.368041,0.631959
2328,0,0,0,1,1,1,0,0,0,0,0,0,0,1,0,0,33,True,2,False,True,149,11,2016,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0.368041,0.631959
2370,0,0,0,1,1,1,1,0,0,0,0,0,0,0,0,0,33,True,2,False,True,149,11,2016,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0.368041,0.631959
2326,0,0,0,1,1,1,0,0,0,0,0,0,0,0,1,0,33,True,2,False,True,149,11,2016,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0.368041,0.631959
2372,0,0,0,1,1,1,1,0,0,0,0,0,0,0,1,0,33,True,2,False,True,149,11,2016,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0.368041,0.631959
2373,0,0,0,1,1,1,1,0,0,0,0,0,0,1,0,0,33,True,2,False,True,149,11,2016,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0.368041,0.631959
2324,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,33,True,2,False,True,149,11,2016,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0.368041,0.631959
2342,0,0,0,1,1,1,0,0,0,0,1,0,0,0,1,0,33,True,2,False,True,149,11,2016,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0.370052,0.629948


In [ ]:
# Prepare LogReg pipeline

logreg_pipeline = Pipeline([
    ('logreg', LogisticRegression(penalty='l1', C=1e5)),
])


In [ ]:
# Grid search

param_grid = [
        {'logreg__penalty': ['l1','l2'],
         'logreg__C': [0.00001,0.0001,0.001,0.01,0.1,1,10]
        },
    ]

logreg_grid_search = GridSearchCV(logreg_pipeline, param_grid, cv=3, refit=True)

logreg_grid_search.fit(X_train, y_train)

logreg_final_model = logreg_grid_search.best_estimator_

print(confusion_matrix(y_test, logreg_final_model.predict(X_test)))

print(accuracy_score(y_test, logreg_final_model.predict(X_test)))

In [ ]:
# Prepare Gradient boost pipeline

gboost_pipeline = Pipeline([
    ('gboost', GradientBoostingClassifier(learning_rate=0.05, subsample=0.5, max_depth=6, n_estimators=10)),
])

In [ ]:
# Grid search

param_grid = [
        {'gboost__n_estimators': [1,10,100,1000,10000],
         'gboost__learning_rate' : [0.001,0.01,0.05,0.1,0.5],
         'gboost__subsample' : [0.1,0.5,1.0],
         'gboost__max_depth': [1,3,5,10,20,50,100]
        },
    ]

gboost_grid_search = GridSearchCV(gboost_pipeline, param_grid, cv=3, refit=True)

gboost_grid_search.fit(X_train, y_train)

gboost_final_model = gboost_grid_search.best_estimator_

print(confusion_matrix(y_test, gboost_final_model.predict(X_test)))

print(accuracy_score(y_test, gboost_final_model.predict(X_test)))

In [ ]:
# Precision / Recall graph


def plot_precision_recall(y_test, y_pred, y_prob):
    average_precision = average_precision_score(y_test, y_pred)
    precision, recall, _ = precision_recall_curve(y_test, y_prob)
    print("{} Precision at {} Recall".format(precision[recall>0.8].max(), 0.8))

    step_kwargs = ({'step': 'post'})
    plt.step(recall, precision, color='b', alpha=0.2, where='post')
    plt.fill_between(recall, precision, alpha=0.2, color='b', **step_kwargs)

    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.ylim([0.0, 1.05])
    plt.xlim([0.0, 1.0])
    plt.title('2-class Precision-Recall curve: AP={0:0.2f}'.format(average_precision))

y_pred = pipeline.predict(X_test)
y_prob = pipeline.predict_proba(X_test)[:,1]
plot_precision_recall(y_test, y_pred, y_prob)

In [ ]:
# Roc curve

fpr, tpr, threshold = roc_curve(y_test, y_prob)
roc_auc = auc(fpr, tpr)
plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

In [ ]:
# Feature importance

pd.Series(rf.feature_importances_, X_train.columns).sort_values(ascending=False).head(20)